In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from skimage import data, color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte
import pandas as pd
from PIL import Image
#from pyzbar import pyzbar
import cv2
from skimage.filters import threshold_otsu
from skimage import exposure


import numpy as np
from skimage.io import imread, imsave
from skimage import exposure
#from skimage.exposure import match_histograms
import numpy as np
import matplotlib.pyplot as plt
import glob
from skimage import data, color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte

import zbar
import cv2
from skimage.filters import threshold_otsu
from skimage import exposure
import os
from scipy.ndimage import rotate
#from pyzbar.pyzbar import decode
from scipy.ndimage.filters import gaussian_filter

np.set_printoptions(threshold=np. inf)
#from BasicDeblurAhmad import BlindDeblur

KeyboardInterrupt: 

In [ ]:
def template_match_qr(full_image,template_passed):
    img2 = full_image.copy()
    template = cv2.imread(template_passed,0)
    print(type(template))
    w, h = template.shape[::-1]
    # All the 6 methods for comparison in a list
    methods = [ 'cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED' , 'cv2.TM_CCORR',
                'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']
    print('template shape is ',template.shape)
    print('img2 shape is ', img2.shape)
    #template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY) #template i uploaded only has 1 dimension. fix this. that is why we error
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    for meth in methods:
        img = img2.copy()
        method = eval(meth)
        # Apply template Matching
        img.astype(np.uint8)
        template.astype(np.uint8)

        res = cv2.matchTemplate(img,template,method)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
        # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
        if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
            top_left = min_loc
        else:
            top_left = max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        
        cv2.rectangle(img,top_left, bottom_right, 255, 20)
        plt.subplot(121),plt.imshow(res,cmap = 'gray')
        plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
        plt.subplot(122),plt.imshow(img,cmap = 'gray')
        plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
        plt.suptitle(meth)
        plt.show()
    

In [ ]:
#so loading in the specific csv to the image works now. work on making sure it places the right square
def CorrectBoxCropping(image_folder,folder_location_csv,deblurred_images):
    detected_slice_count=0
    im_path = sorted(glob.glob(image_folder)) #path to the jpg images folder

    detection_count=0 #detection_count keeps track of 
    csv_of_qr=np.array([[0,0]]) #cvs of qr is where we store csv first column is name of image, second column is decoded message
    csv_of_qr.shape=(1,2)
    for im in (im_path): #here is where we will loop for each photo
        #this part below will be for finding where the box lies
        
        
        
        my_im = im.split('/')[-1]
        name_of_image_csv=my_im.split('.')[0]+'.csv'
    
        print('Loading image: ',my_im)
 

        #im = Image.open(im)
        im = cv2.imread(im)[...,::-1]
        image_1 = np.array(im) 
        height_of_im,width_of_im,color_of_im=image_1.shape
        
        #this code below finds where the box should be and crops it out
        increment_counter=0

        
        csv_names=os.listdir(folder_location_csv)
        #this checks if a label exists. If it doesn't we skip
        if(name_of_image_csv in csv_names):
            labels_csv =pd.read_csv(folder_location_csv+name_of_image_csv, sep=',',header=None)
        else:
            continue
        col_we_looking_at=labels_csv[0].values[0]
        my_split=col_we_looking_at.split()  #my split is a list of each entry in the specific column which is a string. it turns string into a list
        x_center=float(my_split[1])*width_of_im
        y_center=float(my_split[2])*height_of_im
        width=float(my_split[3])*width_of_im
        height=float(my_split[4])*width_of_im
        x_start=int(x_center-.5*width)
        y_start=int(y_center-.5*height)
        x_end=int(x_center+.5*width)
        y_end=int(y_center+.5*height)
        #cv2.rectangle(image_1,(x_start, y_start),
                      #(x_end, y_end),
                      #(220, 0, 0), 50)
        
        #this code below will crop the suit out correctly
        
        buffer=0
        xs = max(x_start-buffer,0)
        ys = max(y_start-buffer,0)
        xe = min(x_end+buffer,image_1.shape[1])
        ye = min(y_end+buffer,image_1.shape[0])

        new_image = np.array(image_1)[ys:ye,xs:xe,:] #this line does cropping
  
        # the lines below blur the image but are unneeded now
    
    
        #new_image_gaussian_blur= cv2.cvtColor(new_image,cv2.COLOR_BGR2GRAY)
        #new_image_gaussian_blur = gaussian_filter(new_image_gaussian_blur, sigma=4)
        #plt.title('gaussian blurred image')
        #plt.imshow(new_image_gaussian_blur,cmap='gray')
        #plt.show()


        
        #contrasting part ends
        
        '''
        This part below handles the decrypting
        '''
        did_it_detect_in_rotate=0
        it_decoded=False
        for i in range(8): #I used 8 since I want to rotate the image 8 times by 45 degrees so it scans thw qr code upright
            new_image_rotated=new_image
            new_image_rotated= cv2.cvtColor(new_image_rotated,cv2.COLOR_BGR2GRAY)

            new_image_rotated=rotate(new_image_rotated,angle=45*i)
            try:
                #qrCodeDetector = cv2.QRCodeDetector()
                #decodedText, points, _ = qrCodeDetector.detectAndDecode(new_image_rotated)
                zbar_scanner = zbar.Scanner()
                decodedText = zbar_scanner.scan(new_image_rotated)
                #print('unenhanced decoded text is',decodedText)

            except:
                continue
            if len(decodedText) > 0:
                print('Text Decoded!')
                plt.title('qr code detected without any modification')
                plt.imshow(new_image_rotated,cmap='gray')
                plt.show()                
                detection_count +=1
                did_it_detect_in_rotate=1
                #add to csv file
                csv_of_qr=np.vstack((csv_of_qr,np.array([my_im,decodedText]))) #this adds it to the csv
                it_decoded=True
                break
        if(did_it_detect_in_rotate==1):
            continue
        
        print('Unenhanced images detected: ',detection_count)
        increment_counter+=1

        
        
        
        
        #this if statement below does the deblurred try, deblurred images should already be cropped and deblurred beforehand
        if (it_decoded!=True):
            did_it_detect_in_rotate=0
            it_decoded=False
            if '*.JPG' in deblurred_images:
                deblurred_images=deblurred_images.replace('*.JPG','')
            deblurred_image=deblurred_images+my_im
            deblurred_image_array = cv2.imread(deblurred_image)[...,::-1]  

            plt.title('deblurred image is')
            plt.imshow(deblurred_image_array,cmap='gray')
            plt.show()
            for i in range(8): #again we rotate like before and try to read it
                deblurred_image_rotated_array=deblurred_image_array
                deblurred_image_rotated_array= cv2.cvtColor(deblurred_image_rotated_array,cv2.COLOR_BGR2GRAY)

                deblurred_image_rotated_array=rotate(deblurred_image_rotated_array,angle=45*i)
                try:
                    #qrCodeDetector = cv2.QRCodeDetector()
                    #decodedText, points, _ = qrCodeDetector.detectAndDecode(new_image_rotated)
                    zbar_scanner = zbar.Scanner()
                    decodedText = zbar_scanner.scan(deblurred_image_rotated_array)
                    #print('unenhanced decoded text is',decodedText)

                except:
                    continue
                if len(decodedText) > 0:
                    print('Text Decoded!')
                    plt.title('deblurred required for qr code to be read')
                    plt.imshow(deblurred_image_rotated_array,cmap='gray')
                    plt.show()                
                    #print('decodedText is',decodedText)
                    detection_count +=1
                    #detected_slice_count+=1
                    did_it_detect_in_rotate=1
                    #add to csv file
                    csv_of_qr=np.vstack((csv_of_qr,np.array([my_im,decodedText])))
                    it_decoded=True
                    break
        if(did_it_detect_in_rotate==1):
            continue
        #my_im is the name and .jpg
        

        
        
        
        
        
        
        
        
        
        
        
        '''
     #this part tries out basic deblurring   
        if (it_decoded!=True):
            did_it_detect_in_rotate=0
            it_decoded=False
            cropped_images='cropped_folder/*.JPG'
            if '*.JPG' in cropped_images:
                cropped_images=cropped_images.replace('*.JPG','')
            cropped_image=cropped_images+my_im
            cropped_image_array = cv2.imread(cropped_image)[...,::-1]  

            cropped_image_array=BlindDeblur(cropped_image_array)
            for i in range(8):
                cropped_image_rotated_array=cropped_image_array
                #cropped_image_rotated_array= cv2.cvtColor(cropped_image_rotated_array,cv2.COLOR_BGR2GRAY) 

                cropped_image_rotated_array=rotate(cropped_image_rotated_array,angle=45*i)

                

                try:
                    #qrCodeDetector = cv2.QRCodeDetector()
                    #decodedText, points, _ = qrCodeDetector.detectAndDecode(new_image_rotated)
                    zbar_scanner = zbar.Scanner()
                    decodedText = zbar_scanner.scan(cropped_image_rotated_array)
                    #print('unenhanced decoded text is',decodedText)

                except:
                    continue
                if len(decodedText) > 0:
                    print('Text Decoded!')
                    plt.title('qr code detected in Blinddeblurred')
                    plt.imshow(cropped_image_rotated_array,cmap='gray')
                    plt.show()                
                    #print('decodedText is',decodedText)
                    detection_count +=1
                    #detected_slice_count+=1
                    did_it_detect_in_rotate=1
                    #add to csv file
                    csv_of_qr=np.vstack((csv_of_qr,np.array([my_im,decodedText])))
                    it_decoded=True
                    break
        if(did_it_detect_in_rotate==1):
            continue
            
            
        '''    
            
            
            
            
            
            
        
        
        
        
        #this part below will try to detect rectangle and try to decode it
        if (it_decoded!=True): 
            cropping_rectangle_return=RectangularDetection2(new_image)
            print('cropping_rectangle_return is',cropping_rectangle_return)
            if(cropping_rectangle_return!=0):
                csv_of_qr=np.vstack((csv_of_qr,np.array([my_im,cropping_rectangle_return])))
                detected_slice_count=detected_slice_count+1
            print('Total Number of Slices Decoded',detected_slice_count)
            #template_match_qr(new_image,'Datasets/template qr code/DSC_1032_cropped.jpg')
            #RectangularDetection(new_image,'Datasets/template qr code/DSC_1032_cropped.jpg')
    print('Total Number of images Decoded',detected_slice_count+detection_count)
    print('final csv is',csv_of_qr)  
    print('final csv shape is',csv_of_qr)
    return csv_of_qr
    



In [ ]:
def RectangularDetection2(full_image_array):
    image=full_image_array
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) 
    close=gray
    num_rows_close,num_cols_close=close.shape
    percentile=np.percentile(close,80) #60 works best here
    print('all colors below this value turn black to help detect white',percentile)
    
    for i in range(num_rows_close):
        for j in range(num_cols_close):
            if(close[i,j]<percentile):
                close[i,j]=0
    plt.title('image after turning white and black is ')
    plt.imshow(close,cmap='gray')
    plt.show()
    #thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,51,9)
    #uncomment above line for thresh
    # Fill rectangular contours
    cnts = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #change to thresh, or close for normal
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        cv2.drawContours(close, [c], -1, (255,255,255), -1) #change this to thresh too, or close for normal

    # Morph open
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
    opening = cv2.morphologyEx(close, cv2.MORPH_OPEN, kernel, iterations=4) #change to thresh, originally 4 iterations, used 1 for numbers i sent joe
    
    # Draw rectangles
    cnts = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) #change to opening for the thresh thing
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    plt.title('image after image processing is')
    plt.imshow(opening,cmap='gray')
    plt.show()
    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        if(y>.2*num_cols_close and w>.2*num_rows_close):
            
            #this takes the box and performs qr code detection
            sliced_image=full_image_array[y:y+h,x:x+w]
            #sliced_image=BlindDeblur(sliced_image)
            plt.title('rectangle detected is')
            plt.imshow(sliced_image,cmap='gray')
            plt.show()

            for i in range(8): # we rotate here again for all 8 orientations
                sliced_image_rotated=rotate(sliced_image,angle=45*i)
                sliced_image_rotated=sliced_image_rotated.astype(np.uint8)

                #qrCodeDetector = cv2.QRCodeDetector()
                #decodedText, points, _ = qrCodeDetector.detectAndDecode(sliced_image)
                
                zbar_scanner = zbar.Scanner()
                sliced_image_rotated= cv2.cvtColor(sliced_image_rotated,cv2.COLOR_BGR2GRAY) #blind deblur returns a grayscale so commented out 
                decodedText = zbar_scanner.scan(sliced_image_rotated)
                print(decodedText)
                if len(decodedText) > 0:
                    return decodedText
                    break
                
            
            '''
            #this is the code for showing the rectangles
            rect = cv2.minAreaRect(c)
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            cv2.drawContours(image,[box],0,(36,255,12),30)
            '''
            
        #cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 30)
    print(opening.shape)
    opening.astype('float')
    #plt.title('thresh')
    #plt.imshow(thresh,cmap='gray')
    #plt.show()
    #plt.title('opening')
    #plt.imshow(opening,cmap='gray')
    #plt.show()
    plt.title('qr code not detected')
    plt.imshow(image,cmap='gray')
    plt.show()
    return 0
    #cv2.imshow('thresh', thresh)
    #cv2.imshow('opening', opening)
    #cv2.imshow('image', image)


In [ ]:
#CorrectBoxCropping('Datasets/csv_bboxes/*.csv','Datasets/LL_109-110/LL_109-110/NikonD780/*.JPG')
csv_109_110=CorrectBoxCropping('Datasets/LL_109-110/LL_109-110/NikonD780/*.JPG','Datasets/csv_bboxes/','deblurred_directory/*.JPG')
csv_115_116=CorrectBoxCropping('Datasets/LL_115-116/LL_115-116/NikonD780/*.JPG','Datasets/csv_bboxes/','deblurred_directory/*.JPG')
csv_119_120=CorrectBoxCropping('Datasets/LL_119-120/LL_119-120/NikonD7000_18mm/*.JPG','Datasets/csv_bboxes/','deblurred_directory/*.JPG')
csv_121_122=CorrectBoxCropping('Datasets/LL_121-122/LL_121-122/NikonD7000_18mm/*.JPG','Datasets/csv_bboxes/','deblurred_directory/*.JPG')
csv_125_126=CorrectBoxCropping('Datasets/LL_125-126/LL_125-126/NikonD7000_18mm/*.JPG','Datasets/csv_bboxes/','deblurred_directory/*.JPG')

In [ ]:
#3499,3500,3831,3446,3447,3448,3449,3450,3451,3452,3499,3500,3501,3502,3503,3504,3831,3832,3833,3834,3835,4056,4057
#print(csv_109_110[0,:])
csv_109_110_copy=np.delete(csv_109_110, (0), axis=0) #deletes that dumb 0,0 in first row. just insert this in regular code
print(csv_109_110_copy)

In [ ]:
#for ll_109-110: 0 slices and 26 unenhanced with zbar-py red channel
#for ll_115-116: 2 slices and 29 unenhanced with zbar-py red channel
#for ll_119-120: 1 slices and 24 unenhanced with zbar-py red channel
#for ll_121-122: 5 slices and 14 unenhanced with zbar-py red channel
#for ll_125-126: 2 slices and 22 unenhanced with zbar-py red channel




#for ll_109-110: 0 slices and 26 unenhanced with zbar-py blue channel
#for ll_115-116: 1 slices and 30 unenhanced with zbar-py blue channel
#for ll_119-120: 4 slices and 18 unenhanced with zbar-py blue channel
#for ll_121-122: 4 slices and 17 unenhanced with zbar-py blue channel
#for ll_125-126: 2 slices and 20 unenhanced with zbar-py blue channel

In [ ]:
#for ll_109-110: 0 slices and 26 unenhanced with zbar-py
#for ll_115-116: 0 slices and 31 unenhanced with zbar-py
#for ll_119-120: 0 slices and 25 unenhanced with zbar-py
#for ll_121-122: 2 slices and 17 unenhanced with zbar-py
#for ll_125-126: 2 slices and 22 unenhanced with zbar-py



#for ll_109-110: 0 slices and 26 unenhanced with zbar-py deblur
#for ll_115-116: 0 slices and 34 unenhanced with zbar-py deblur
#for ll_119-120: 0 slices and 26 unenhanced with zbar-py deblur
#for ll_121-122: 2 slices and 17 unenhanced with zbar-py deblur
#for ll_125-126: 2 slices and 22 unenhanced with zbar-py deblur

In [ ]:
#for ll_109-110: My rectangles decoded 16, while the annotations only did 5 went up to 20 once i rotated first image and combined
#for ll_115-116: My rectangles decoded 5, while the annotations only did 5 went up to 13 once i rotated first image and combined 
#for ll_119-120: My rectangles decoded 12, while the annotations only did 4 went up to 18 once i rotated first image and combined
#for ll_121-122: My rectangles decoded 11, while the annotations only did 5 went up to 13 once i rotated first image and combined
#for ll_125-126: My rectangles decoded 12, while the annotations only did 4 up to 14 once i rotated first image and combined

In [ ]:
#take images I could already read blur them and use deblurgan on them, and try to read it with deblurgan. use it on those initial 125 

In [ ]:
def BlindDeblur(array_passed):
    import numpy as np
    import matplotlib.pyplot as plt

    from skimage import color, data, restoration
    rng = np.random.default_rng()
    astro=color.rgb2gray(array_passed)
    #astro = color.rgb2gray(data.astronaut())
    from scipy.signal import convolve2d as conv2
    psf = np.ones((5, 5)) / 25
    astro = conv2(astro, psf, 'same')
    astro += 0.1 * astro.std() * rng.standard_normal(astro.shape)

    deconvolved, _ = restoration.unsupervised_wiener(astro, psf)

    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 5),
                           sharex=True, sharey=True)

    plt.gray()

    ax[0].imshow(astro, vmin=deconvolved.min(), vmax=deconvolved.max())
    ax[0].axis('off')
    ax[0].set_title('Data')

    ax[1].imshow(deconvolved)
    ax[1].axis('off')
    ax[1].set_title('Self tuned restoration')

    fig.tight_layout()

    plt.show()
    return deconvolved